In [55]:
import codecs

positive = [] 
negative = []
posneg = []
dic = [('./KOSELF_pos.txt', './KOSELF_neg.txt'),
       ("./data/positive_words_self.txt","./data/negative_words_self.txt"),
      ('./data/sum_pos.txt','./data/sum_neg.txt')]

i = int(input('koself : 1, blog : 2, sum : 3'))-1


pos = codecs.open(dic[i][0], 'rb', encoding='UTF-8') 

while True: 
    line = pos.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
        
    positive.append(line) 
    posneg.append(line) 
        
pos.close() 

neg = codecs.open(dic[i][1], 'rb', encoding='UTF-8') 
while True: 
    line = neg.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
    
    negative.append(line) 
    posneg.append(line) 
    
neg.close()


koself : 1, blog : 2, sum : 33


In [46]:
import pandas as pd
import numpy as np
import pymysql
import re

In [47]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}
media_list = {'아시아경제_' : 'asia_',
             '매일경제_': 'maeil_'}

df_list = []

for n,m in media_list.items():
    for i,j in com_list.items():

        db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

        cursor = db.cursor(pymysql.cursors.DictCursor)
        sql = "select * from "+ m +"news_craw_" + j +" where length(date) = 10"
        
        cursor.execute(sql)

        result = cursor.fetchall()

        # 데이터 프레임으로 변경
        globals() ['df_'+ m + j ] = pd.DataFrame(result)
        print('df_'+ m + j)
        df_list.append(globals() ['df_'+ m + j ])
        
        db.close()

df_asia_000660
df_asia_005380
df_asia_005930
df_asia_051910
df_asia_068270
df_maeil_000660
df_maeil_005380
df_maeil_005930
df_maeil_051910
df_maeil_068270


In [48]:
df_total = pd.concat(df_list)

In [49]:
df_total

,st_n,st_cd,news,date,title,url,text
0,sk하이닉스,000660,아시아경제,2018122715,"SK하이닉스, 우수 엔지니어 60세 넘어도 일할 수 있다",http://view.asiae.co.kr/news/view.htm?idxno=20...,2020년에는 상대평가제도도 폐지…소통강화 위해 호칭도 'TL'로 통일 [아시아경제...
1,sk하이닉스,000660,아시아경제,2018123110,"오션브릿지, SK하이닉스와 12억 규모 반도체 장비제조 공급계약 체결",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 오주연 기자] \r\t\t\t\t\t\t\...
2,sk하이닉스,000660,아시아경제,2018122816,"SK하이닉스, 우수 엔지니어 정년 없앤다",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t인사제도 파격적 개편…최태원 회장의 딥체인지 실현 [아...
3,sk하이닉스,000660,아시아경제,2018122607,"[클릭 e종목]""SK하이닉스, 내년 반도체 수요 증가율 상저하고…목표가↓""",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 송화정 기자]신한금융투자는 26일 \r\t...
4,sk하이닉스,000660,아시아경제,2018122707,"[클릭 e종목]""SK하이닉스, 4Q영업익 전망치 하향""",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 오주연 기자] 하나금융투자가 27일 \r\...
...,...,...,...,...,...,...,...
3216,셀트리온,068270,매일경제,2021092309,"""미국 중국 동시 악재라니""…동학개미 악소리 난다, 연휴 뒤 코스피 급락",http://news.mk.co.kr/newsRead.php?no=907885&ye...,\n\n\n[사진 출처 = 연합뉴스]\n 중국 헝다그룹 유동성 위기 등 추석 연휴에...
3217,셀트리온,068270,매일경제,2021092317,"디지털 강자 매경, 카카오 블록체인 클레이튼과 손잡다",http://news.mk.co.kr/newsRead.php?no=910365&ye...,\n\n\n올해로 22회째를 맞은 세계지식포럼에서 로버트 주브린 화성학회 회장이 2...
3218,셀트리온,068270,매일경제,2021092715,"코스피 외국인 기관 쌍끌이 매수…8.40포인트↑, 3133.64 장 마감",http://news.mk.co.kr/newsRead.php?no=919526&ye...,\n\n\n[사진 출처 = 연합뉴스]\n 중국 부동산 재벌 헝다그룹의 파산 우려에도...
3219,셀트리온,068270,매일경제,2021092414,"휴마시스, 셀트리온과 273억원 규모 코로나19 항원진단키트 공급 계약",http://news.mk.co.kr/newsRead.php?no=912835&ye...,현장형 신속검사 기업 휴마시스가 셀트리온과 273억원 규모 코로나19 항원진단키트 ...


In [90]:
test_df = pd.read_csv('./sentiment/2018_lg.csv')

In [91]:
test_df

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2018010118,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 중국발 수요 증가에 힘입어 새해 '전기차 랠리'에 대한 기대감이 확산되...,2018-01-02,18,407000,415000,406500,411000,153739,0.014815,1,0,중국 수요 증가 새해 전기차 랠리 대한 대감 확산 한국 거래소 전기차 배터리 핵심 ...
1,LG화학,51910,매일경제,2018010204,"LG ,마곡사이언스파크 완공 R&D강화",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n지난해 미국 라스베이거스에서 열린 `CES 2017`에서 LG전자가 올레...,2018-01-02,4,407000,415000,406500,411000,153739,0.014815,1,0,지난해 미국 라스베이거스 전자 레드 사이니 이용 너비 높이 길이 규모 레드 터널 관...
2,LG화학,51910,매일경제,2018010204,"[다시 뛰는 재계] 차세대 먹을거리 차곡차곡 ""4차 산업혁명 우리가 주도""",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n인천국제공항 제2터미널에서 항공기가 떠오르고 있다. 무술년(戊戌年)을 맞...,2018-01-02,4,407000,415000,406500,411000,153739,0.014815,1,0,인천 국제공항 터미널 항공기 무술년 맞이 우리 기업 보호무역 주의 파고 고유 금리 ...
3,LG화학,51910,매일경제,2018010210,"[신년사] 박진수 LG화학 부회장 ""사업구조·사업방식·조직문화 바꿔야""",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n박진수 LG화학 부회장. [사진 제공 = LG화학]\n 박진수 LG화학 ...,2018-01-02,10,407000,415000,406500,411000,153739,0.014815,1,0,박진수 화학 부회장 사진 제공 화학 박진수 화학 부회장 기존 방식 감히 우리 사업 ...
4,LG화학,51910,매일경제,2018010215,"코스피, 새해 첫 거래일 상승 마감…코스닥 800 고지 돌파",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 코스피가 2018년 무술년 첫 거래일 상승 마감했다. 코스닥은 1%대 ...,2018-01-03,15,404000,413000,404000,409500,103370,-0.003650,-1,0,코스피 무술년 거래 상승 마감 코스닥 강세 위쪽 거래 코스닥 위쪽 마감 지난 이후 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,LG화학,51910,아시아경제,2018122615,"코스피, 글로벌 증시 급락에 2028선 마감…하락폭은 1%대",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 오주연 기자] 글로벌 증시 급락 여파로 2...,2018-12-27,15,359000,359500,345500,347000,172656,-0.016997,-1,0,아시아 경제 오주연 기자 글로벌 증시 급락 여파 코스피 하락 후퇴 코스피 거래 대비...
1706,LG화학,51910,아시아경제,2018122709,美 훈풍에 반등한 韓 증시...코스닥 1.39%↑,http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 이정민 기자] 국내 증시가 27일 장 초반...,2018-12-27,9,359000,359500,345500,347000,172656,-0.016997,-1,0,아시아 경제 이정민 기자 국내 증시 초반 상승 전날 급등 미국 증시 덕분 투자 심리...
1707,LG화학,51910,아시아경제,2018122716,"코스피, 美호재 vs 배당락일 효과에 소폭 상승 마감",http://view.asiae.co.kr/news/view.htm?idxno=20...,문호남 기자 munonam@[아시아경제 이정민 기자] 한국 증시가 27일 미국발 훈...,2018-12-28,16,349000,352500,345500,347000,107893,0.000000,-1,0,호남 기자 아시아 경제 이정민 기자 한국 증시 미국 훈풍 배당 락일 효과 치면 눈치...
1708,LG화학,51910,아시아경제,2018123007,"'AA' SK가스, 석유화학 사업다각화 'A+' E1에 판정승",http://view.asiae.co.kr/news/view.htm?idxno=20...,[아시아경제 문채석 기자]\r\t\t\t\t\t\t\t\tSK가스SK가스018670...,2019-01-02,7,351000,353500,335500,337000,227694,-0.028818,-1,0,아시아 경제 채석 기자 가스 가스 코스피 증권 정보 현재 전일 대비 거래 전일 장마...


In [92]:
test_df = test_df[['st_n', 'st_cd', 'news', 'date', 'title', 'Change', 'Tokenization']]

In [93]:
test_df

,st_n,st_cd,news,date,title,Change,Tokenization
0,LG화학,51910,매일경제,2018-01-02,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,0.014815,중국 수요 증가 새해 전기차 랠리 대한 대감 확산 한국 거래소 전기차 배터리 핵심 ...
1,LG화학,51910,매일경제,2018-01-02,"LG ,마곡사이언스파크 완공 R&D강화",0.014815,지난해 미국 라스베이거스 전자 레드 사이니 이용 너비 높이 길이 규모 레드 터널 관...
2,LG화학,51910,매일경제,2018-01-02,"[다시 뛰는 재계] 차세대 먹을거리 차곡차곡 ""4차 산업혁명 우리가 주도""",0.014815,인천 국제공항 터미널 항공기 무술년 맞이 우리 기업 보호무역 주의 파고 고유 금리 ...
3,LG화학,51910,매일경제,2018-01-02,"[신년사] 박진수 LG화학 부회장 ""사업구조·사업방식·조직문화 바꿔야""",0.014815,박진수 화학 부회장 사진 제공 화학 박진수 화학 부회장 기존 방식 감히 우리 사업 ...
4,LG화학,51910,매일경제,2018-01-03,"코스피, 새해 첫 거래일 상승 마감…코스닥 800 고지 돌파",-0.003650,코스피 무술년 거래 상승 마감 코스닥 강세 위쪽 거래 코스닥 위쪽 마감 지난 이후 ...
...,...,...,...,...,...,...,...
1705,LG화학,51910,아시아경제,2018-12-27,"코스피, 글로벌 증시 급락에 2028선 마감…하락폭은 1%대",-0.016997,아시아 경제 오주연 기자 글로벌 증시 급락 여파 코스피 하락 후퇴 코스피 거래 대비...
1706,LG화학,51910,아시아경제,2018-12-27,美 훈풍에 반등한 韓 증시...코스닥 1.39%↑,-0.016997,아시아 경제 이정민 기자 국내 증시 초반 상승 전날 급등 미국 증시 덕분 투자 심리...
1707,LG화학,51910,아시아경제,2018-12-28,"코스피, 美호재 vs 배당락일 효과에 소폭 상승 마감",0.000000,호남 기자 아시아 경제 이정민 기자 한국 증시 미국 훈풍 배당 락일 효과 치면 눈치...
1708,LG화학,51910,아시아경제,2019-01-02,"'AA' SK가스, 석유화학 사업다각화 'A+' E1에 판정승",-0.028818,아시아 경제 채석 기자 가스 가스 코스피 증권 정보 현재 전일 대비 거래 전일 장마...


In [94]:
j = 0
label = [0] * len(test_df)

my_dic = {"title":[], "label":label}

titles = test_df.title
titles = titles.str.replace('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“▲》◆▶◇■]','')
titles = titles.str.replace('[\n\r\t\\\]','')
titles = titles.reset_index().drop('index',1).title

for df in range(len(test_df)):    
    txt_data = titles[df]
    
    my_dic['title'].append(txt_data) 

    for i in range(len(posneg)): 
        posflag = False 
        negflag = False 

        if i < (len(positive)-1): 
            if txt_data.find(posneg[i]) != -1: 
                posflag = True 
                print(i, "positive?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

        if i > (len(positive)-2): 
            if txt_data.find(posneg[i]) != -1: 
                negflag = True 
                print(i, "negative?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

    if posflag == True: 
        label[j] = 1 
        # print("positive", j) 

    elif negflag == True: 
        label[j] = -1 
        # print("negative", j) 

    elif negflag == False and posflag == False: 
        label[j] = 0 
        # print("objective", j) 

    j = j + 1
                
my_dic['label'] = label
my_df = pd.DataFrame(my_dic)

19 positive? 테스트 :  23 비교단어 :  수혜 인덱스 :  19
52 positive? 테스트 :  29 비교단어 :  돌파 인덱스 :  52
52 positive? 테스트 :  25 비교단어 :  돌파 인덱스 :  52
52 positive? 테스트 :  21 비교단어 :  돌파 인덱스 :  52
84 positive? 테스트 :  32 비교단어 :  최고 인덱스 :  84
84 positive? 테스트 :  0 비교단어 :  최고 인덱스 :  84
137 negative? 테스트 :  10 비교단어 :  하락 인덱스 :  137
2 positive? 테스트 :  24 비교단어 :  강세 인덱스 :  2
62 positive? 테스트 :  4 비교단어 :  1위 인덱스 :  62
52 positive? 테스트 :  32 비교단어 :  돌파 인덱스 :  52
21 positive? 테스트 :  6 비교단어 :  실현 인덱스 :  21
78 positive? 테스트 :  12 비교단어 :  증가 인덱스 :  78
137 negative? 테스트 :  15 비교단어 :  하락 인덱스 :  137
84 positive? 테스트 :  2 비교단어 :  최고 인덱스 :  84
84 positive? 테스트 :  6 비교단어 :  최고 인덱스 :  84
84 positive? 테스트 :  7 비교단어 :  최고 인덱스 :  84
52 positive? 테스트 :  21 비교단어 :  돌파 인덱스 :  52
44 positive? 테스트 :  6 비교단어 :  호황 인덱스 :  44
84 positive? 테스트 :  7 비교단어 :  최고 인덱스 :  84
84 positive? 테스트 :  0 비교단어 :  최고 인덱스 :  84
19 positive? 테스트 :  20 비교단어 :  수혜 인덱스 :  19
137 negative? 테스트 :  16 비교단어 :  하락 인덱스 :  137
2 positive? 테스트 :  17 비교단어 :  강세 인덱스 

In [95]:
my_df

,title,label
0,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,1
1,LG 마곡사이언스파크 완공 RD강화,0
2,다시 뛰는 재계 차세대 먹을거리 차곡차곡 4차 산업혁명 우리가 주도,0
3,신년사 박진수 LG화학 부회장 사업구조·사업방식·조직문화 바꿔야,0
4,코스피 새해 첫 거래일 상승 마감코스닥 800 고지 돌파,1
...,...,...
1705,코스피 글로벌 증시 급락에 2028선 마감하락폭은 1대,-1
1706,美 훈풍에 반등한 韓 증시코스닥 139↑,0
1707,코스피 美호재 vs 배당락일 효과에 소폭 상승 마감,0
1708,AA SK가스 석유화학 사업다각화 A E1에 판정승,0


In [96]:
my_df['ch'] = test_df['Change']

In [98]:
my_df['ch'] = np.where(my_df['ch'] > 0.01, 1,
                       np.where(my_df['ch'] < 0.01,-1,0))

In [99]:
(my_df['ch'] == my_df['label']).sum() / len(my_df)

0.15029239766081873

In [97]:
my_df.label.value_counts()

 0    1217
-1     249
 1     244
Name: label, dtype: int64

In [67]:
my_df.to_csv('./data/title_labeling_total.csv', index=False)

In [68]:
my_df.title[0]

'SK하이닉스 우수 엔지니어 60세 넘어도 일할 수 있다'

In [ ]:
하이닉스 --> 하이닉스 좋다 --> 기사 긍정 --> 기사의 분류결과 <- 을 앙상블을하는 거
하이닉스 --> 긍정판단 --> 주식이 올르거라고 판단한 거라 볼수 있다 <- 기사들마다 정확도를 낼때 쓸수있는 변수


하이닉스 --> 감성사전 구축 (레이블 : 주식등락) --> 주식등락이랑 비교를 하면서 정확도를 본것

보 팅(LSTM,머신러닝) + 기사긍부정 + 강화학습 --> 분류 

In [69]:
import FinanceDataReader as fdr